In [223]:
#Import Library
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import spotipy
from matplotlib import style
from spotipy import util
from spotipy.oauth2 import SpotifyClientCredentials 
loc='G:\\Projects\\MIR\\DataMining'

In [224]:
#Set up Connection 
client_id = 'a90f2a20aea54d5781d4cc7a38ab6694' #Need to create developer profile
client_secret = 'b8810bc6ac1d4877b78c45a8654ebf18'
username = 'f0uthgq19mkamht0gno58wkt0' #Store username
scope = 'playlist-read-collaborative'
redirect_uri='http://localhost/'
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, 
client_secret=client_secret)#Create manager for ease
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
token = util.prompt_for_user_token(username, scope, client_id, 
client_secret, redirect_uri)

if token:
 sp = spotipy.Spotify(auth=token)
else:
 print("Can't get token for", username)

In [225]:
#Connect to the best playlist, like ever 
playlist_id = 'spotify:playlist:6X4iVRG9PAyNT7gRUAZEMb'
playlist = sp.user_playlist(username,playlist_id)

In [226]:
#Extract list of songs/tracks
tracks = playlist["tracks"];
songs = tracks["items"];

In [227]:
track_ids = []
track_names = []

In [228]:
for i in range(0, len(songs)):
 if songs[i]['track']['id'] != None: # Removes  local tracks, if any
    track_ids.append(songs[i]['track']['id'])
    track_names.append(songs[i]['track']['name'])

In [229]:
def user_playlist_tracks_full(spotify_connection, user, playlist_id=None, fields=None, market=None):
    """ Get full details of the tracks of a playlist owned by a user.
        https://developer.spotify.com/documentation/web-api/reference/playlists/get-playlists-tracks/

        Parameters:
            - user - the id of the user
            - playlist_id - the id of the playlist
            - fields - which fields to return
            - market - an ISO 3166-1 alpha-2 country code.
    """

    # first run through also retrieves total no of songs in library
    response = spotify_connection.user_playlist_tracks(user, playlist_id, fields=fields, limit=100, market=market)
    results = response["items"]
    

    # subsequently runs until it hits the user-defined limit or has read all songs in the library
    while len(results) < response["total"]:
        response = spotify_connection.user_playlist_tracks(user, playlist_id, fields=fields, limit=100, offset=len(results), market=market)
        results.extend(response["items"]) 
    return(results)


In [230]:
results=user_playlist_tracks_full(sp,username,playlist_id)


In [231]:
results[0]['track']['uri']


'spotify:track:3NZJlJemX3mzjf56MqC5ML'

In [232]:
track_uri=[]
raw_playlist_data=[]
for i in range(0,len(results)):
    if songs[i]['track']['id'] != None: # Removes  local tracks, if any
        play=results[i]
        raw_playlist_data.append(play['track'])
        track_uri.append(play['track']['uri'])
               

In [233]:
for i in range(len(raw_playlist_data)):
    del raw_playlist_data[i]['available_markets']
    del raw_playlist_data[i]['disc_number']
    del raw_playlist_data[i]['episode']
    del raw_playlist_data[i]['external_ids']
    del raw_playlist_data[i]['external_urls']
    del raw_playlist_data[i]['is_local']
    del raw_playlist_data[i]['preview_url']
    del raw_playlist_data[i]['type']
    del raw_playlist_data[i]['album']
    del raw_playlist_data[i]['artists']

In [234]:
raw_playlist_data[1]

{'duration_ms': 303200,
 'explicit': False,
 'href': 'https://api.spotify.com/v1/tracks/7w5cxTEzp1rfV3KCy0Bd5N',
 'id': '7w5cxTEzp1rfV3KCy0Bd5N',
 'name': 'Home',
 'popularity': 71,
 'track': True,
 'track_number': 6,
 'uri': 'spotify:track:7w5cxTEzp1rfV3KCy0Bd5N'}

In [235]:
def audio_features(element):
    #Add new key-values to store audio features
    element['acousticness'] = []
    element['danceability'] = []
    element['energy'] = []
    element['instrumentalness'] = []
    element['liveness'] = []
    element['loudness'] = []
    element['speechiness'] = []
    element['tempo'] = []
    element['valence'] = []
    #element['popularity'] = []
    #create a track counter
    
    features = sp.audio_features(element['uri'])
        
    #Append to relevant key-value
    element['acousticness'].append(features[0]['acousticness'])
    element['danceability'].append(features[0]['danceability'])
    element['energy'].append(features[0]['energy'])
    element['instrumentalness'].append(features[0]['instrumentalness'])
    element['liveness'].append(features[0]['liveness'])
    element['loudness'].append(features[0]['loudness'])
    element['speechiness'].append(features[0]['speechiness'])
    element['tempo'].append(features[0]['tempo'])
    element['valence'].append(features[0]['valence'])
    #popularity is stored elsewhere
    #pop = sp.track(i)
    #element['popularity'].append(pop['popularity'])

In [236]:
for i in raw_playlist_data:
    audio_features(i)

In [237]:
raw_playlist_data[0]

{'duration_ms': 278573,
 'explicit': False,
 'href': 'https://api.spotify.com/v1/tracks/3NZJlJemX3mzjf56MqC5ML',
 'id': '3NZJlJemX3mzjf56MqC5ML',
 'name': 'Forever',
 'popularity': 72,
 'track': True,
 'track_number': 1,
 'uri': 'spotify:track:3NZJlJemX3mzjf56MqC5ML',
 'acousticness': [0.0368],
 'danceability': [0.672],
 'energy': [0.82],
 'instrumentalness': [0.000188],
 'liveness': [0.184],
 'loudness': [-4.456],
 'speechiness': [0.0459],
 'tempo': [120.005],
 'valence': [0.438]}

In [243]:
type(raw_playlist_data[0]['acousticness'])

list

In [239]:
df=pd.DataFrame([raw_playlist_data][0])

In [240]:
print(len(df))
final_df = df.sort_values('popularity', ascending=False).drop_duplicates('name').sort_index()
print(len(final_df))

32
32


In [241]:
final_df.to_csv(loc+"\\Songs.csv")